In [5]:
import math
import elevation as eio
from pathlib import Path

An equivalent function to this notebook is in load.py/load_from_eio()

In [7]:
#lat, lon = 44.5451545715, 4.2165341377
lat, lon =  35.18057, -111.65288

def bounds_from_distance(km):
    dlat = (km/2) / 111.11
    dlon = dlat / math.cos(math.radians(lat))
    return [lon - dlon, lat - dlat, lon + dlon, lat + dlat]

bounds = bounds_from_distance(50)

In [20]:
datapath = Path.cwd()/'data/MNS/'
tiff_file = datapath/'test.tiff'
eio.clip(bounds=bounds, output=tiff_file, product='SRTM3')

make: Entering directory '/home/nicod/.cache/elevation/SRTM3'
make: Nothing to be done for 'download'.
make: Leaving directory '/home/nicod/.cache/elevation/SRTM3'
make: Entering directory '/home/nicod/.cache/elevation/SRTM3'
make: Nothing to be done for 'all'.
make: Leaving directory '/home/nicod/.cache/elevation/SRTM3'
make: Entering directory '/home/nicod/.cache/elevation/SRTM3'
cp SRTM3.vrt SRTM3.52f25525dd9f42489b8dd6e0dad9b496.vrt
make: Leaving directory '/home/nicod/.cache/elevation/SRTM3'
make: Entering directory '/home/nicod/.cache/elevation/SRTM3'
gdal_translate -q -co TILED=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=2 -projwin -111.92816587770554 35.4055722500225 -111.37759412229445 34.955567749977504 SRTM3.52f25525dd9f42489b8dd6e0dad9b496.vrt /media/nicod/Documents/D4G/pyronear/lidar/data/MNS/test.tiff
rm -f SRTM3.52f25525dd9f42489b8dd6e0dad9b496.vrt
make: Leaving directory '/home/nicod/.cache/elevation/SRTM3'


In [21]:
from osgeo import gdal

xyz_path = (datapath/'test.xyz').as_posix()
ds = gdal.Open(tiff_file.as_posix())
xyz = gdal.Translate(xyz_path, ds)

In [22]:
import pandas as pd

test = pd.read_csv(xyz_path, sep=' ', names=['x', 'y', 'z'])
test.head()

,x,y,z
0,-111.927917,35.405417,2210
1,-111.927083,35.405417,2210
2,-111.926250,35.405417,2216
3,-111.925417,35.405417,2218
4,-111.924583,35.405417,2224


In [23]:
from pyproj import Transformer
import numpy as np
projector = Transformer.from_crs("EPSG:4326", "EPSG:2154")
def project_row(row):
        return np.array(projector.transform(row['y'], row['x'])+(row['z'],))
test = test.apply(project_row, axis=1, result_type='broadcast')
# for i in range(len(test)):
#     test.iloc[i] = np.array(projector.transform(test.at[i, 'y'], test.at[i,'x'])+(test.at[i,'z'],))
test.head()


,x,y,z
0,-6.545501e+06,1.181633e+07,2210.0
1,-6.545492e+06,1.181625e+07,2210.0
2,-6.545483e+06,1.181618e+07,2216.0
3,-6.545474e+06,1.181610e+07,2218.0
4,-6.545465e+06,1.181602e+07,2224.0


In [14]:
test.to_csv(xyz_path, sep=' ', header=False, index=False)